# Fashion Chatbot

The fashion industry is constantly evolving, with new trends and styles emerging every season. Keeping up with the latest fashion can be overwhelming, especially for those who are not immersed in the industry. This is where a fashion chatbot comes into play. By leveraging natural language processing and machine learning techniques, a fashion chatbot can assist users in navigating the vast world of fashion, providing personalized recommendations, answering queries, and offering styling advice.

## Traditional Gpt

In [ ]:
!pip install --upgrade openai
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (

In [ ]:
import openai
api_key = input("Please enter your OpenAI API key: ")
openai.api_key = api_key

**Standard Chat bot**

In [ ]:
def set_open_params(
    model="gpt-3.5-turbo",
    temperature=0.5,
    max_tokens=400,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
):
    """ set openai parameters"""

    openai_params = {}

    openai_params['model'] = model
    openai_params['temperature'] = temperature
    openai_params['max_tokens'] = max_tokens
    openai_params['top_p'] = top_p
    openai_params['frequency_penalty'] = frequency_penalty
    openai_params['presence_penalty'] = presence_penalty
    return openai_params

In [ ]:
def get_completion(params, messages):
    response = openai.chat.completions.create(
        model = params['model'],
        messages = messages,
        temperature = params['temperature'],
        max_tokens = params['max_tokens'],
        top_p = params['top_p'],
        frequency_penalty = params['frequency_penalty'],
        presence_penalty = params['presence_penalty'],
    )
    return response

In [ ]:
params = set_open_params()

In [ ]:
prompt = "I want some stylish winter wear for women?"
messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
print(response.choices[0].message.content)

Here are some stylish winter wear options for women:

1. Oversized knit sweaters - Perfect for layering and keeping warm during the colder months.

2. Faux fur coats - A trendy and cozy option for staying stylish in the winter.

3. Puffer jackets - A practical and fashionable choice for staying warm in the snow.

4. Knit beanies - A cute and functional accessory to keep your head warm.

5. Thigh-high boots - A stylish and versatile option for adding some edge to your winter outfits.

6. Cashmere scarves - A luxurious and soft accessory to keep you warm and stylish.

7. Turtleneck sweaters - A classic and chic option for staying warm and stylish in the winter.

8. Leather gloves - A sleek and sophisticated accessory to keep your hands warm in the cold weather.

9. Wool blend coats - A timeless and elegant option for staying warm and stylish in the winter.

10. Sherpa-lined denim jackets - A trendy and cozy option for adding some texture to your winter wardrobe.


In [ ]:
prompt = "I want floral dresses within $100"
messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
print(response.choices[0].message.content)

Here are some options for floral dresses under $100:

1. ASOS Design Petite Floral Print Midi Dress - $50
2. H&M Floral Wrap Dress - $35
3. Forever 21 Floral Ruffle Mini Dress - $25
4. Old Navy Floral Tiered Swing Dress - $40
5. Target A New Day Floral Maxi Dress - $30

These are just a few options, but there are many more affordable floral dresses available at various retailers. Happy shopping!


It is clear from the result that the responses from the model are generic.An organisation planning to implement a chatbot within their system should fine tune the model to give more relevant responses.

## Fine tuning the model using RAG

### Fine tuning using SentenceTransformer

In [ ]:
import pandas as pd

df = pd.read_csv("Styles.csv")

In [ ]:
def row_to_sentence(row):
  data= f"The {row['productDisplayName']} is a {row['articleType']} in {row['baseColour']} color, designed for {row['usage']} during {row['season']}."\
  f"It is {row['Fit']} fit and fabric composition is {row['Fabric Composition']}"\
  f"The product belongs to {row['subCategory']} category under {row['masterCategory']}"\
  f"It is priced at ${row['Price']} with a discount of {row['Discount']}%. Available sizes: XS({row['XS']}), S({row['S']}), M({row['M']}, L({row['L']}, XL({row['XL']},XXL({row['XXL']}"\
  f" Rated {row['Average Rating']}/5 from {row['Number of Ratings']} reviews. Suitable for {row['Target Audience']}."\
  f"Wash care is {row['Wash Care']} for the material {row['Material']}.It was manufactured in {row['Country of Manufacture']} by designer {row['Designer']} and is part of {row['Collection']} collection."

  if row['Eco-Friendly'] == 'Yes':
    data += "The product is eco-friendly."
  if row['Sustainable Certification'] != 'None':
    data+= "It has {row['Sustainable Certification']} sustainable certification."

  return data


In [ ]:
df['description'] = df.apply(row_to_sentence, axis=1)

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
df['embeddings'] = df['description'].apply(lambda x: model.encode(x, convert_to_tensor=True))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
from sentence_transformers import util
import torch

def modify_query_for_apparel(query):
    if 'wear' in query:
        query += " +apparel +clothing"
    return query

def semantic_search(query, df, top_k=3):
    modified_query = modify_query_for_apparel(query)
    query_embedding = model.encode(modified_query, convert_to_tensor=True)
    scores = util.semantic_search(query_embedding, torch.stack(df['embeddings'].tolist()))[0]
    top_indices = [score['corpus_id'] for score in scores[:top_k]]
    return df.iloc[top_indices]

import openai

def generate_response(retrieved_products,query):
    context = "Based on the query :"+query+", here are the top products:\n"
    context += "\n".join(retrieved_products['description'].tolist())
    context += "\nFormulate a proper response for user using this for the query?"
    prompt_text =[{"role": "system", "content": "You are a helpful fashion assistant. Answer the fashion query."},
                  {"role":"user","content":context}]
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=prompt_text,
        max_tokens=400,
        temperature=0.5,
        top_p =1,
        frequency_penalty = 0,
        presence_penalty = 0
    )
    return response.choices[0].message.content

def handle_user_query(query):
    top_products = semantic_search(query, df)
    return generate_response(top_products,query)


In [ ]:
user_query = "Suggest me some superhero themed party dresses for women"
response = handle_user_query(user_query)
print(response)

For a superhero themed party, here are some dress options for women that you might like:

1. **United Colors of Benetton Women Purple Dress**:
   - Color: Purple
   - Style: Casual, perfect for Summer
   - Fit: Regular fit
   - Fabric: 71% Cotton
   - Price: $78.0 (5.1% discount)
   - Sizes: XS, S, M, L, XL, XXL
   - Rating: 4.2/5 from 380 reviews
   - Suitable for: Young Adults
   - Wash care: Delicate wash
   - Manufacturer: Chanel (China)
   - Collection: Streetwear
   - Sustainable certification: Yes

2. **Tonga Women Purple Dress**:
   - Color: Purple
   - Style: Casual, ideal for Fall
   - Fit: Regular fit
   - Fabric: 100% Silk
   - Price: $96.0 (1.5% discount)
   - Sizes: XS, S, M, L, XL, XXL
   - Rating: 3.6/5 from 98 reviews
   - Suitable for: Young Adults
   - Wash care: Dry clean only
   - Manufacturer: Armani (China)
   - Collection: Couture
   - Sustainable certification: Not specified

3. **Vogue Women Purple Sunglasses**:
   - Color: Purple
   - Style: Casual, great for

In [ ]:
query = "What are some stylish watches for women in silver within 100?"
response = handle_user_query(query)
print(response)

Based on your query for stylish silver watches for women under $100, here are some recommendations:

1. Timex Women's Easy Reader Date Expansion Band Watch - This classic silver watch features an easy-to-read dial and an expansion band for a comfortable fit. Priced under $100, it offers both style and functionality.

2. Anne Klein Women's Swarovski Crystal Accented Silver-Tone Watch - This elegant watch is adorned with Swarovski crystals, adding a touch of sparkle to your wrist. It is priced affordably and perfect for adding a touch of glamour to any outfit.

3. Casio Women's Classic Quartz Watch - A timeless silver watch from Casio that is both stylish and budget-friendly. With a simple yet chic design, this watch is versatile and suitable for everyday wear.

These options should provide you with stylish silver watches for women within your budget.


In [ ]:
query = "I want some stylish summer wear for women?"
response = handle_user_query(query)
print(response)

For stylish summer wear options for women, here are some recommendations:

1. **Disney Kids Girl's Summer Fun Fair Yellow Teen Kidswear T-shirt**
   - Color: Yellow
   - Fit: Regular
   - Fabric: 84% Cotton, 15% Others
   - Price: $72.0 (16.8% discount)
   - Available Sizes: XS, S, M, L, XL, XXL
   - Rating: 3.8/5 from 137 reviews
   - Suitable for: Young Adults
   - Wash Care: Hand wash
   - Manufacturer: Armani (USA)
   - Collection: Sportswear
   - Sustainable Certification: Yes

2. **W Women Red Dress Kurta**
   - Color: Red
   - Fit: Regular
   - Fabric: 100% Silk
   - Price: $84.0 (26.8% discount)
   - Available Sizes: XS, M, L, XL, XXL
   - Rating: 0.7/5 from 94 reviews
   - Suitable for: Young Adults
   - Wash Care: Dry clean only
   - Manufacturer: Calvin Klein (Vietnam)
   - Collection: Sportswear
   - Sustainable Certification: Not specified

3. **Disney Kids Girl's Summer Fun Fair Pink Teen Kidswear T-shirt**
   - Color: Pink
   - Fit: Regular
   - Fabric: 100% Silk
   - Pr

In [ ]:
query = "I want some stylish winter wear for women?"
response = handle_user_query(query)
print(response)

For stylish winter wear options for women, I recommend the Forever New Women Long Black Jacket. This jacket is perfect for casual wear during the winter season. It is made of 97% Cotton and 3% Others, providing both comfort and style. The black color makes it versatile and easy to pair with different outfits. 

With a discount of 7.3%, the jacket is priced at $72.0. It comes in various sizes ranging from XS to XXL, catering to different body types. Although it has received a rating of 1.6/5 from 35 reviews, it is still a popular choice among young adults.

The Forever New Women Long Black Jacket is eco-friendly and has sustainable certification. It is recommended to delicate wash this jacket due to its cotton material. This fashionable piece was manufactured in Vietnam by the renowned designer Gucci and is part of the Resort collection.

Stay warm and stylish this winter with the Forever New Women Long Black Jacket!


In [ ]:
query = "Suggest me some stylish resort wear for women and some accessories also that go with it"
response = handle_user_query(query)
print(response)

For a stylish resort wear look for women, I recommend the following outfit and accessories:

- Pair the W Women Red Dress, a beautiful red silk kurta, with the Carlton London Women Casual Red Stilettos for a chic and elegant ensemble perfect for a resort setting. The red dress is designed for casual wear during summer, while the red stilettos add a touch of sophistication to the outfit.

- Complete the look with the Stoln Women Black Sunglasses to protect your eyes from the sun in style. These black sunglasses are designed for casual wear during winter and will add a trendy touch to your resort outfit.

- Enhance your overall look with the Boss Women Femme Perfume, a delightful pink perfume and body mist perfect for casual wear during spring. The feminine scent will complement your resort attire beautifully.

- As a bonus, consider adding the Nike Fragrances Women Paradise Perfume and Deo Set to your collection. This fragrance gift set in white color is designed for casual wear during 

In [ ]:
query = "Suggest me some stylish dresses under $20"
response = handle_user_query(query)
print(response)

Based on the information provided, I recommend the following stylish dresses under $20:

1. The Jealous 21 Women Black Dress in Black color, made of 90% Cotton and 9% Others fabric composition, priced at $80.0 with a discount of 22.7%. It is available in various sizes and suitable for Young Adults. This dress is designed for Casual wear during Summer and is eco-friendly. It was manufactured in Italy by designer Calvin Klein and is part of the Couture collection.

2. The 109F Red & White A-Line Dress in Red color, made of 100% Silk fabric composition, priced at $99.0 with a discount of 29.9%. It is available in different sizes and suitable for Young Adults. This dress is designed for Casual wear during Summer and is eco-friendly. It was manufactured in Italy by designer Gucci and is part of the Streetwear collection.

Unfortunately, the dresses mentioned are priced above $20, so they may not fit within your specified budget. You may want to consider exploring other options or sales to f

In [ ]:
#query = "Suggest me some loose fit jeans with rating more than 4 for women under $100"
query = "Suggest me a loose fit jeans priced less than $80 for women and rating more than 4"
response = handle_user_query(query)
print(response)

Based on your query for loose fit jeans priced less than $80 and with a rating of more than 4, I recommend the Pepe Jeans Women Blue Jeans in Regular fit. This pair of jeans is priced at $56.0 with a discount of 36.9%. It has a fabric composition of 81% Polyester, making it suitable for casual wear during the summer. The jeans have received a rating of 4.6/5 from 77 reviews, making it a highly recommended choice. Available sizes include XS, S, M, L, XL, and XXL. The wash care instructions recommend delicate wash for the Polyester material. This eco-friendly product was manufactured in the USA by designer Versace and is part of the Resort collection.


In the last query,we can see that the chatbot is suggesting regular fit jeans despite asking for loose fit jeans.We can try another technique of RAG to get better responses.

### Chatbot using FAISS

In [ ]:
!pip install langchain
!pip install openai
!pip install faiss-gpu
!pip install langchain_experimental
!pip install "langchain[docarray]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 16.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependenc

In [ ]:
!pip install langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.5 MB/s eta 0:00:00


In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory

In [ ]:

from langchain.document_loaders.csv_loader import CSVLoader
from langchain_openai import OpenAIEmbeddings

In [ ]:
import os

api_key = input("Please enter your OpenAI API key: ")

os.environ["OPENAI_API_KEY"] = api_key


print("OPENAI_API_KEY has been set!")

In [ ]:
loader = CSVLoader(file_path="Styles.csv")
documents = loader.load()

embeddings_model = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(documents, embeddings_model)

In [ ]:
# Create conversation chain
llm = ChatOpenAI(temperature=0.5,
                 model_name="gpt-3.5-turbo",
                 max_tokens=400)
memory = ConversationBufferMemory(
memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        memory=memory
        )

In [ ]:
query = "Suggest me some stylish resort wear for women and some accessories also that go with it"
result = conversation_chain({"question": query})
answer = result["answer"]
print(answer)

For stylish resort wear for women, you can consider the following options:

1. **Product:** Shree Women Multi Coloured Jodhpuri Salwar
   - **Price:** $114
   - **Discount:** 47.3%
   - **Season:** Fall
   - **Designer:** Gucci
   - **Country of Manufacture:** Vietnam
   - **Collection:** Resort
   - **Fabric Composition:** 75% Polyester, 5% Others
   - **Fit:** Regular
   - **Clearance:** Yes
   - **Product Launch Date:** 2/22/2012

2. **Product:** Shree Women Charcoal Grey & Sea Green Printed Salwar Suit with Dupatta
   - **Price:** $78
   - **Discount:** 22.9%
   - **Season:** Summer
   - **Designer:** Gucci
   - **Country of Manufacture:** China
   - **Collection:** Resort
   - **Fabric Composition:** 71% Polyester, 6% Others
   - **Fit:** Regular
   - **Clearance:** Yes
   - **Product Launch Date:** 4/18/2012

For accessories that go well with resort wear, you can consider the following:

1. **Product:** Estelle Women Gold Jewellery Set
   - **Price:** $76
   - **Discount:** 12.9%

In [ ]:
query = "I want some stylish summer wear for women?"
result = conversation_chain({"question": query})
answer = result["answer"]
print(answer)

Here are some stylish summer wear options for women based on the provided data:

1. Product: Scullers For Her Women Fashion Summer Black Tunics
   - Type: Tunics
   - Price: $48
   - Material: Cotton
   - Wash Care: Hand wash
   - Season: Fall
   - Designer: Versace
   - Country of Manufacture: Italy
   - Clearance: Yes
   - New Arrivals: Yes
   - Trend Factor: 2

2. Product: 109F Women Multi Coloured Dress
   - Type: Dresses
   - Price: $72
   - Material: Polyester
   - Wash Care: Hand wash
   - Season: Summer
   - Designer: Versace
   - Country of Manufacture: Vietnam
   - Eco-Friendly: Yes
   - Featured in Collections: Yes

3. Product: W Women Tan Kurti
   - Type: Tops
   - Price: $114
   - Material: Polyester
   - Wash Care: Hand wash
   - Season: Summer
   - Designer: Gucci
   - Country of Manufacture: Italy
   - New Arrivals: Yes
   - Trend Factor: 9

These options include tunics, dresses, and kurtis that are suitable for summer wear and come in stylish designs.


In [ ]:

query = "Suggest me a loose fit jeans priced less than $80 for women and rating more than 4"
result = conversation_chain({"question": query})
answer = result["answer"]
print(answer)

I'm sorry, based on the provided information, there isn't a loose fit jeans for women priced under $80 with a rating of more than 4.


In [ ]:
query = "Suggest me some stylish dresses under $100"
result = conversation_chain({"question": query})
answer = result["answer"]
print(answer)

Based on the provided data, here are some stylish dresses under $100:

1. Dress: 109F Women Multi Coloured Dress
   - Price: $72

2. Dress: 109F Red & White A-Line Dress
   - Price: $99 (Discounted to under $100)

3. Dress: 109F Blue A-Line Dress
   - Price: $78

Please note that the prices mentioned above are before any discounts. The Red & White A-Line Dress is discounted to under $100.


## Recommendation from image

In [ ]:
import torch
from torchvision.models import resnet50
from torchvision import transforms
from PIL import Image

imageModel = resnet50(pretrained=True)
imageModel.eval()

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 56.3MB/s]


In [ ]:
import cv2
from PIL import Image

def load_image(img):
    img_t = transform(img)
    return img_t.unsqueeze(0)



In [ ]:
import json
import urllib.request

class_idx_url = "https://storage.googleapis.com/download.tensorflow.org/data/imagenet_class_index.json"
class_idx_path, _ = urllib.request.urlretrieve(class_idx_url)
class_idx = json.load(open(class_idx_path))

# Function to get predictions
def get_predictions(image_tensor):
    with torch.no_grad():
        outputs = imageModel(image_tensor)
        _, predicted = torch.max(outputs, 1)
        class_id = predicted.item()
        return class_idx[str(class_id)][1]

In [ ]:
def generate_query_from_image(user_image):
  image_tensor = load_image(user_image)
  keyword = get_predictions(image_tensor)
  user_query = "Suggest me some options for " + keyword
  return user_query

In [ ]:
def handle_fashion_query(user_query,user_image=None):
  if user_image!=None:
    modified_query = generate_query_from_image( user_image)
  else:
    modified_query = user_query
  result=conversation_chain({"question": modified_query})
  return result["answer"]

In [ ]:
user_prompt ="Look at the image and give me suggestions"
user_image = Image.open("1584.jpg")
result = handle_fashion_query(user_prompt,user_image)
print(result)

Here are some options for sweatshirts based on the provided data:

1. Nike Men Printed White Sweatshirts
2. Nike Men Black Sweatshirt
3. Nike Men Casual Black Sweatshirts
4. Urban Yoga Women Printed Black Sweatshirt

These are some of the sweatshirt options available, each with its unique features and details.


In [ ]:
result = handle_fashion_query("I want floral dresses within $100")
print(result)

Based on the available data, I can suggest the following floral dresses within $100:

1. Product: 109F Blue A-Line Dress
   - Price: $78
   - Base Colour: Blue
   - Season: Summer
   - Material: Cotton
   - Designer: Chanel
   - Country of Manufacture: USA

2. Product: 109F Women Multi Coloured Dress
   - Price: $72
   - Base Colour: Multi
   - Season: Summer
   - Material: Polyester
   - Designer: Versace
   - Country of Manufacture: Vietnam

Please note that these dresses are within the specified price range and feature floral designs.


In [ ]:
user_query = "I want an eco friendly outfit for a barn wedding"
result = handle_fashion_query(user_query)
print(result)

For an eco-friendly outfit for a barn wedding, you could consider the "Mother Earth Women Green Printed Kurta" made of Cotton. It's a green kurta that is both eco-friendly and suitable for an ethnic-themed event like a barn wedding. The fabric composition is 95% Cotton, making it a sustainable choice. Additionally, it was launched in the summer season, which could be perfect for a barn wedding.


In [ ]:
user_query = "Can you suggest me good jeans options for XXL size women"
result = handle_fashion_query(user_query)
print(result)

Based on the available data, here are some good jeans options for XXL size women:

1. Elle Women Black Jeans
   - Price: $80
   - Discount: 18.4%
   - Material: Polyester
   - Fit: Slim
   - Available in XXL size

2. Elle Women Blue Jeans
   - Price: $90
   - Discount: 9%
   - Material: Silk
   - Fit: Loose
   - Available in XXL size

Please note that these options are based on the information provided and may not cover all available options in the market.


In [ ]:
user_query = "What accessories would you suggest for a red party gown"
result = handle_fashion_query(user_query)
print(result)

For a red party gown, you could consider accessorizing with gold earrings to complement the red color. The "Royal Diadem Red Earrings" or "Royal Diadem Magenta Earrings" from the provided data could be good options, as they are designed for women and have a touch of gold that would match well with a red gown. These earrings are suitable for ethnic or formal occasions, depending on the style of the party.


In [ ]:
user_query = "Can you suggest good kidswear for baby boy?"
result = handle_fashion_query(user_query)
print(result)

For good kidswear options for a baby boy, you can consider the following suggestions based on the provided data:

1. **Nike Kids Boys Contemporary Athlete Blue T-shirt**
   - Price: $72
   - Material: Polyester
   - Season: Summer
   - Usage: Sports
   - Available Sizes: XS, S, M, L, XL, XXL
   - Designer: Armani
   - Country of Manufacture: USA
   - Collection: Sportswear
   - New Arrivals: Yes
   - Featured in Collections: Yes

2. **Doodle Boy's Athletic 77 Dept White Kidswear**
   - Price: $68
   - Material: Polyester
   - Season: Fall
   - Usage: Casual
   - Available Sizes: XS, S, M, L, XL, XXL
   - Designer: Calvin Klein
   - Country of Manufacture: India
   - Collection: Sportswear
   - New Arrivals: No
   - Featured in Collections: No

These options provide a mix of sporty and casual wear for baby boys, depending on the occasion and preference.
